## Import Code

In [ ]:
!git clone https://github.com/alinourian/Adaptive-Torch-KWT.git

In [ ]:
cd Adaptive-Torch-KWT/

## Install Requirements

In [ ]:
!pip install -qr requirements.txt

## Download Dataset

In [ ]:
!sh ./download_gspeech_v2.sh ./data/

In [ ]:
!ls data

In [ ]:
!python make_data_list.py -v ./data/validation_list.txt -t ./data/testing_list.txt -d ./data/ -o ./data/ -n 5

## Train Model

In [ ]:
conf_str = """# sample config to run a demo training of 20 epochs

data_root: ./data/
train_list_file: ./data/training_list.txt
val_list_file: ./data/validation_list.txt
test_list_file: ./data/testing_list.txt
label_map: ./data/label_map.json

exp:
    wandb: False
    wandb_api_key: <path/to/api/key>
    proj_name: torch-kwt-1
    exp_dir: ./runs
    exp_name: exp-0.0.1
    device: auto
    log_freq: 20    # log every l_f steps
    log_to_file: True
    log_to_stdout: True
    val_freq: 1    # validate every v_f epochs
    n_workers: 1
    pin_memory: True
    cache: 2 # 0 -> no cache | 1 -> cache wavs | 2 -> cache specs; stops wav augments
    

hparams:
    seed: 0
    batch_size: 512
    n_epochs: 10
    l_smooth: 0.1

    audio:
        sr: 16000
        n_mels: 40
        n_fft: 480
        win_length: 480
        hop_length: 160
        center: False
    
    model:
        name: # if name is provided below settings will be ignored during model creation   
        input_res: [40, 98]
        patch_res: [40, 1]
        num_classes: 35
        mlp_dim: 256
        dim: 64
        heads: 1
        depth: 12
        dropout: 0.0
        emb_dropout: 0.1
        pre_norm: False
        adaptive_model: True

    optimizer:
        opt_type: adamw
        opt_kwargs:
          lr: 0.001
          weight_decay: 0.1
    
    scheduler:
        n_warmup: 10
        max_epochs: 140
        scheduler_type: cosine_annealing

    augment:
        # resample:
            # r_min: 0.85
            # r_max: 1.15
        
        # time_shift:
            # s_min: -0.1
            # s_max: 0.1

        # bg_noise:
            # bg_folder: ./data/_background_noise_/

        spec_aug:
            n_time_masks: 2
            time_mask_width: 25
            n_freq_masks: 2
            freq_mask_width: 7"""

!mkdir -p configs
with open("configs/kwt1_colab.yaml", "w+") as f:
  f.write(conf_str)

In [ ]:
!python train.py --conf configs/kwt1_colab.yaml